In [29]:
import numpy as np
import scipy as sci
import cv2
from numpy import linalg

In [30]:
#ファイルパスを入力してください
dir1 = "d:/images/2-1.JPG"
dir2 = "d:/images/2-2.JPG"

In [31]:
def get_points(img_dir1,img_dir2):
    points1 = []
    points2 = []

    def OnMouseAction1(event,x,y,flags,param):

        if event == cv2.EVENT_LBUTTONDOWN:
            print("from camera1:(%s,%s)"%(x,y))
            points1.append([x,y])
        elif event==cv2.EVENT_RBUTTONDOWN :
            print("camera1 finish")
            cv2.destroyWindow("image1")
            
    def OnMouseAction2(event,x,y,flags,param):

        if event == cv2.EVENT_LBUTTONDOWN:
            print("from camera2:(%s,%s)"%(x,y))
            points2.append([x,y])
        elif event==cv2.EVENT_RBUTTONDOWN :
            print("camera2 finish")
            cv2.destroyWindow("image2")

    img1 = cv2.imread(img_dir1)
    img2 = cv2.imread(img_dir2)
    cv2.namedWindow("image1",cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('image1',OnMouseAction1)     
    cv2.namedWindow("image2",cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('image2',OnMouseAction2)
    cv2.imshow('image1',img1)
    cv2.imshow('image2',img2)
    cv2.waitKey(80000)
    
         
    
    return (np.array(points1),np.array(points2))

In [32]:
def get_mat_M(point1,point2,count):
    M = np.zeros((count,9))
    for i in range(count):
        M[i,0] = point1[i,0] * point2[i,0]
        M[i,1] = point1[i,1] * point2[i,0]
        M[i,2] = point2[i,0]
        M[i,3] = point1[i,0] * point2[i,1]
        M[i,4] = point1[i,1] * point2[i,1]
        M[i,5] = point2[i,1]
        M[i,6] = point1[i,0]
        M[i,7] = point1[i,1]
        M[i,8] = 1.0
    return M

In [33]:
def etimate_F(M):
    mattmat = np.dot(M.T,M)
    eig,eig_vec = linalg.eig(mattmat)
    F = eig_vec.T[-1].reshape(3,3)
    print(eig)
    print(F)
    
    _,eig_list_e1 = linalg.eig(np.dot(F.T,F))
    e1 = eig_list_e1.T[-1]/eig_list_e1.T[-1][2]
    _,eig_list_e2 = linalg.eig(np.dot(F,F.T))
    e2 = eig_list_e2.T[-1]/eig_list_e2.T[-1][2]
    print((e1,e2))
    return(F,e1,e2)

In [34]:
def draw_lines(point1,point2,e1,e2,dir1,dir2):
    one = np.ones((len(point1),1),"float")
    point1_ = np.concatenate((point1,one),axis=1)
    point2_ = np.concatenate((point2,one),axis=1)
    line1 = []
    line2 = []
    
    for p1 , p2 in zip(point1_,point2_):
        line1.append(np.cross(e1,p1))
        line2.append(np.cross(e2,p2))
        
    i = 0
    cv2.namedWindow('epilines1')
    img1 = cv2.imread(dir1)
    for l1,p1 in zip(line1,point1):
        i += 1
        aa = tuple(p1.tolist())
        print("circle")
        cv2.circle(img1, aa, 15, (255,0,16), -1)
        cv2.putText(img1,'%s'%i,(p1[0],p1[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(0,0,255),9)
        print("lines")
        cv2.line(img1, (15, int(-l1[2] / l1[1])),(img1.shape[1], int(-(l1[2] + l1[0] * img1.shape[1]) / l1[1])),(0,255,0),5)
    cv2.imshow('epilines1',img1)
    cv2.imwrite("D:/2-1_lines.jpg", img1)
    cv2.waitKey(0)
    
    j = 0
    cv2.namedWindow('epilines2')
    img2 = cv2.imread(dir2)
    for l2,p2 in zip(line2,point2):
        j += 1
        bb = tuple(p2.tolist())
        cv2.circle(img2, bb, 15, (255,0,16), -1)
        cv2.putText(img2,'%s'%j,(p2[0],p2[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(0,0,255),9)
        cv2.line(img2, (0, int(-l2[2] / l2[1])),(img2.shape[1], int(-(l2[2] + l2[0] * img2.shape[1]) / l2[1])), (0,255,0),5)
    cv2.imshow('epilines2',img2)
    cv2.imwrite("D:/2-2_lines.jpg", img2)
    cv2.waitKey(0)


In [ ]:
def draw_lines2(point1,point2,e1,e2,dir1,dir2):
    one = np.ones((len(point1),1),"float")
    point1_ = np.concatenate((point1,one),axis=1)
    point2_ = np.concatenate((point2,one),axis=1)
    line1 = []
    line2 = []
    
    for p1 , p2 in zip(point1_,point2_):
        line1.append(np.cross(e1,p1))
        line2.append(np.cross(e2,p2))
        
    i = 0
    cv2.namedWindow('epilines1')
    img1 = cv2.imread(dir1)
    for l1,p1 in zip(line1,point1):
        i += 1
        aa = tuple(p1.tolist())
        print("circle")
        cv2.circle(img1, aa, 15, (0,255,16), -1)
        cv2.putText(img1,'%s'%i,(p1[0],p1[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(255,0,0),9)
        print("lines")
        cv2.line(img1, (15, int(-l1[2] / l1[1])),(img1.shape[1], int(-(l1[2] + l1[0] * img1.shape[1]) / l1[1])),(255,0,0),5)
    cv2.imshow('epilines1',img1)
    cv2.imwrite("D:/2-1_linesexp.jpg", img1)
    cv2.waitKey(0)
    
    j = 0
    cv2.namedWindow('epilines2')
    img2 = cv2.imread(dir2)
    for l2,p2 in zip(line2,point2):
        j += 1
        bb = tuple(p2.tolist())
        cv2.circle(img2, bb, 15, (0,255,16), -1)
        cv2.putText(img2,'%s'%j,(p2[0],p2[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(255,0,0),9)
        cv2.line(img2, (0, int(-l2[2] / l2[1])),(img2.shape[1], int(-(l2[2] + l2[0] * img2.shape[1]) / l2[1])), (255,0,0),5)
    cv2.imshow('epilines2',img2)
    cv2.imwrite("D:/2-2_linesexp.jpg", img2)
    cv2.waitKey(0)


In [41]:
#ここから
a,b = get_points(dir1,dir2)

In [36]:
M = get_mat_M(a,b,len(a))

In [39]:
F,e1,e2 = etimate_F(M)
draw_lines(a,b,e1,e2,dir1,dir2)

[ 9.67168287e+14  1.40040933e+14  1.40953329e+12  3.19455643e+10
  1.45635003e+05  1.47256312e+03  4.16905236e+02  4.73599556e+01
 -1.81577903e-12]
[[ 1.75089241e-08  7.34015604e-08 -2.04670180e-04]
 [-9.20187045e-08  1.90270545e-07 -2.15295529e-04]
 [ 1.62928561e-04 -6.13113160e-04  9.99999755e-01]]
(array([-1.26489484e+03,  1.29488699e+03,  1.00000000e+00]), array([-2.75579597e+04,  3.08426741e+04,  1.00000000e+00]))
circle
lines
circle
lines
circle
lines
circle
lines
circle
lines
circle
lines
circle
lines
circle
lines


In [26]:
#ランク修正
U,sigma,VT = linalg.svd(F)
sigma_mat = np.array([[sigma[0],0,0],[0,sigma[1],0],[0,0,0]])
F_2 = np.dot(np.dot(U,sigma_mat),VT)

In [28]:
F_2

array([[-4.97935043e-08,  4.33984300e-07, -5.81036097e-04],
       [-9.11996663e-08,  1.99805750e-07, -5.60078269e-03],
       [-4.60424139e-04,  4.84880397e-03,  9.99972285e-01]])